In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Mato Grosso - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [5]:
data = pd.read_csv('2003_mo_model_input_MT.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Mato Grosso - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso - Desemprego,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - value,Mato Grosso - Consumo de Cimento (t)
0,2003-1,50.917240,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.297178,0.745147,6.084564e+07,2.670588e+06,16.831332,5.118946e+07,0.331800,47.470
1,2003-2,50.851586,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.291317,0.745308,6.092768e+07,2.672688e+06,16.835967,5.121614e+07,0.333615,45.387
2,2003-3,50.970549,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.285457,0.745469,6.100973e+07,2.674788e+06,16.840602,5.124282e+07,0.334864,44.907
3,2003-4,51.049978,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.279596,0.745630,6.109177e+07,2.676888e+06,16.845237,5.126950e+07,0.336048,45.467
4,2003-5,51.567529,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.273735,0.745791,6.117381e+07,2.678988e+06,16.849872,5.129619e+07,0.336447,56.246
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,119.544326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.600607,192.533
236,2022-9,118.223448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.598965,183.895
237,2022-10,117.524152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.596866,180.525
238,2022-11,116.430559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.594246,161.683


In [6]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Mato Grosso - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso - Desemprego,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - value
0,-1.582351,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,1.216878,-2.486031,-1.594683,-2.079312,-1.524159,-1.566766,-1.015691
1,-1.585655,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,1.204107,-2.439296,-1.579788,-2.049593,-1.511861,-1.551683,-0.977876
2,-1.579669,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,1.191337,-2.392562,-1.564893,-2.019873,-1.499562,-1.536601,-0.951860
3,-1.575671,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,1.178566,-2.345827,-1.549998,-1.990154,-1.487263,-1.521519,-0.927186
4,-1.549625,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,1.165795,-2.299093,-1.535104,-1.960435,-1.474964,-1.506437,-0.918874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.683341,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.462024,0.554708,1.263263,0.134707,1.180043,1.228996,0.893787
188,0.683776,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.475003,0.507932,1.258638,0.140184,1.177055,1.225355,0.914063
189,0.711014,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.487981,0.461157,1.254013,0.145662,1.174067,1.221714,0.954337
190,0.715019,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.500960,0.414381,1.249388,0.151139,1.171079,1.218073,0.995597


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      51.972
1      40.729
2      54.446
3      51.788
4      58.702
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Mato Grosso - Consumo de Cimento (t), Length: 240, dtype: float64

In [8]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Mato Grosso - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso - Desemprego,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - value
0,-1.582351,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,1.216878,-2.486031,-1.594683,-2.079312,-1.524159,-1.566766,-1.015691
1,-1.585655,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,1.204107,-2.439296,-1.579788,-2.049593,-1.511861,-1.551683,-0.977876
2,-1.579669,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,1.191337,-2.392562,-1.564893,-2.019873,-1.499562,-1.536601,-0.951860
3,-1.575671,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,1.178566,-2.345827,-1.549998,-1.990154,-1.487263,-1.521519,-0.927186
4,-1.549625,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,1.165795,-2.299093,-1.535104,-1.960435,-1.474964,-1.506437,-0.918874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.341278,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,-0.104237,1.450474,1.217726,0.274296,1.163104,1.185879,1.167738
158,0.322813,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,-0.191347,1.427979,1.221923,0.260732,1.162467,1.188073,1.148285
159,0.303209,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-0.278458,1.405484,1.226120,0.247169,1.161831,1.190267,1.127808
160,0.275768,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-0.365569,1.382988,1.230317,0.233605,1.161195,1.192461,1.108543


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0       51.972
1       40.729
2       54.446
3       51.788
4       58.702
        ...   
157     65.293
158     90.338
159     83.188
160    107.230
161    105.397
Name: Mato Grosso - Consumo de Cimento (t), Length: 162, dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Mato Grosso - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Mato Grosso - Desemprego,Mato Grosso - IDH,Mato Grosso - PIB - Estadual,Mato Grosso - PIB - Construção Civil,Mato Grosso - PIB - Per Capita,Mato Grosso - PIB - Preços de Mercado,Mato Grosso - value
162,0.327220,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,-0.539790,1.337998,1.238711,0.206478,1.159923,1.196848,1.067331
163,0.378712,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,-0.626901,1.315502,1.242908,0.192915,1.159287,1.199042,1.046219
164,0.415005,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,-0.714012,1.293007,1.247105,0.179352,1.158651,1.201236,1.024290
165,0.428855,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,-0.801123,1.270512,1.251302,0.165788,1.158014,1.203430,1.016200
166,0.436119,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,-0.888234,1.248017,1.255499,0.152225,1.157378,1.205623,1.007780
167,0.457915,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,-0.975344,1.225521,1.259696,0.138661,1.156742,1.207817,0.999210
168,0.462907,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,-1.062455,1.203026,1.263893,0.125098,1.156106,1.210011,0.992238
169,0.467673,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,-1.088182,1.176285,1.266539,0.122704,1.159844,1.213717,0.985213
170,0.491605,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,-1.113908,1.149545,1.269184,0.120309,1.163582,1.217423,0.978151
171,0.482102,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,-1.139635,1.122804,1.271830,0.117915,1.167320,1.221129,0.975477


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    120.055
163    125.769
164    112.904
165    123.201
166    112.725
167     90.828
168     96.360
169     86.444
170     94.285
171     98.986
172     88.072
173    141.010
174    122.652
175    142.145
176    121.124
177    130.503
178    104.115
179     90.690
180    102.685
181     96.144
182    102.197
183    106.712
184    124.057
185    124.625
186    133.116
187    144.310
188    140.357
189    152.769
190    124.038
191     95.054
Name: Mato Grosso - Consumo de Cimento (t), dtype: float64

In [12]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [13]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [16]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1400218057, 5067504, 702131231, 2461182336, 1249198093, 3671784685, 4156832109, 352242301, 478062164, 1370490601, 670041207, 3616518440, 3758750456, 1043582868, 1158710163, 521754126, 1191840380, 1825004737, 4043393830, 2790677958, 2917524552, 3159585003, 881316346, 310717172, 4165869612]


Step: 0 ___________________________________________
val_loss: 101.48622131347656
winner_seed: 1400218057


Step: 1 ___________________________________________
val_loss: 89.9061508178711
winner_seed: 5067504


Step: 2 ___________________________________________
val_loss: 97.91637420654297


Step: 3 ___________________________________________
val_loss: 68.97753143310547
winner_seed: 2461182336


Step: 4 ___________________________________________
val_loss: 249.74295043945312


Step: 5 ___________________________________________
val_loss: 69.87838745117188


Step: 6 ___________________________________________
val_loss: 278.2451477050781


Step: 7 ___________________________________________
val_loss: 8

2023-10-14 17:02:44.506971: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 264.751953125


Step: 17 ___________________________________________


2023-10-14 17:03:41.213740: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 90.18218231201172


Step: 18 ___________________________________________
val_loss: 81.07556915283203


Step: 19 ___________________________________________
val_loss: 63.881412506103516
winner_seed: 2790677958


Step: 20 ___________________________________________
val_loss: 76.33917236328125


Step: 21 ___________________________________________
val_loss: 83.08082580566406


Step: 22 ___________________________________________
val_loss: 85.22663879394531


Step: 23 ___________________________________________


2023-10-14 17:08:34.260957: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-14 17:08:34.314841: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 89.31342315673828


Step: 24 ___________________________________________
val_loss: 84.21521759033203


final_seed: 2790677958


In [18]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 26ms/step - loss: 7362.6143 - val_loss: 14538.1152
Epoch 2/10000
5/5 [==============================] - 0s 6ms/step - loss: 8320.8037 - val_loss: 7749.1543
Epoch 3/10000
5/5 [==============================] - 0s 6ms/step - loss: 6777.6675 - val_loss: 6114.8140
Epoch 4/10000
5/5 [==============================] - 0s 7ms/step - loss: 5791.9404 - val_loss: 4493.5400
Epoch 5/10000
5/5 [==============================] - 0s 6ms/step - loss: 5228.0483 - val_loss: 3800.3176
Epoch 6/10000
5/5 [==============================] - 0s 7ms/step - loss: 4191.5664 - val_loss: 1470.5496
Epoch 7/10000
5/5 [==============================] - 0s 6ms/step - loss: 1905.9834 - val_loss: 1753.1826
Epoch 8/10000
5/5 [==============================] - 0s 6ms/step - loss: 2477.3142 - val_loss: 371.7469
Epoch 9/10000
5/5 [==============================] - 0s 6ms/step - loss: 357.7549 - val_loss: 244.3091
Epoch 10/10000
5/5 [==============================] - 0s

5/5 [==============================] - 0s 6ms/step - loss: 152.3326 - val_loss: 136.0126
Epoch 80/10000
5/5 [==============================] - 0s 6ms/step - loss: 146.9256 - val_loss: 98.7045
Epoch 81/10000
5/5 [==============================] - 0s 6ms/step - loss: 172.4497 - val_loss: 127.5226
Epoch 82/10000
5/5 [==============================] - 0s 6ms/step - loss: 143.4392 - val_loss: 131.5808
Epoch 83/10000
5/5 [==============================] - 0s 6ms/step - loss: 133.3283 - val_loss: 126.9979
Epoch 84/10000
5/5 [==============================] - 0s 6ms/step - loss: 127.8052 - val_loss: 122.7912
Epoch 85/10000
5/5 [==============================] - 0s 6ms/step - loss: 124.6247 - val_loss: 184.8777
Epoch 86/10000
5/5 [==============================] - 0s 6ms/step - loss: 250.0753 - val_loss: 136.6172
Epoch 87/10000
5/5 [==============================] - 0s 6ms/step - loss: 166.8985 - val_loss: 125.8850
Epoch 88/10000
5/5 [==============================] - 0s 7ms/step - loss: 142.62

5/5 [==============================] - 0s 6ms/step - loss: 133.6372 - val_loss: 109.6189
Epoch 158/10000
5/5 [==============================] - 0s 6ms/step - loss: 134.8777 - val_loss: 117.7261
Epoch 159/10000
5/5 [==============================] - 0s 6ms/step - loss: 148.4240 - val_loss: 102.2123
Epoch 160/10000
5/5 [==============================] - 0s 6ms/step - loss: 144.7990 - val_loss: 107.2571
Epoch 161/10000
5/5 [==============================] - 0s 6ms/step - loss: 164.2989 - val_loss: 175.0839
Epoch 162/10000
5/5 [==============================] - 0s 6ms/step - loss: 166.4443 - val_loss: 205.9170
Epoch 163/10000
5/5 [==============================] - 0s 6ms/step - loss: 236.9656 - val_loss: 105.8753
Epoch 164/10000
5/5 [==============================] - 0s 6ms/step - loss: 168.9181 - val_loss: 171.9391
Epoch 165/10000
5/5 [==============================] - 0s 7ms/step - loss: 179.7061 - val_loss: 158.7379
Epoch 166/10000
5/5 [==============================] - 0s 7ms/step - lo

5/5 [==============================] - 0s 6ms/step - loss: 189.5333 - val_loss: 187.5339
Epoch 236/10000
5/5 [==============================] - 0s 5ms/step - loss: 209.5379 - val_loss: 144.6026
Epoch 237/10000
5/5 [==============================] - 0s 6ms/step - loss: 200.6584 - val_loss: 130.0162
Epoch 238/10000
5/5 [==============================] - 0s 5ms/step - loss: 186.4392 - val_loss: 148.5745
Epoch 239/10000
5/5 [==============================] - 0s 6ms/step - loss: 224.3144 - val_loss: 237.3400
Epoch 240/10000
5/5 [==============================] - 0s 5ms/step - loss: 218.7155 - val_loss: 172.5709
Epoch 241/10000
5/5 [==============================] - 0s 5ms/step - loss: 217.3755 - val_loss: 190.3451
Epoch 242/10000
5/5 [==============================] - 0s 6ms/step - loss: 185.5774 - val_loss: 143.6536
Epoch 243/10000
5/5 [==============================] - 0s 5ms/step - loss: 180.8171 - val_loss: 135.4354
Epoch 244/10000
5/5 [==============================] - 0s 6ms/step - lo

5/5 [==============================] - 0s 6ms/step - loss: 161.4910 - val_loss: 205.9286
Epoch 314/10000
5/5 [==============================] - 0s 5ms/step - loss: 138.2291 - val_loss: 170.8841
Epoch 315/10000
5/5 [==============================] - 0s 6ms/step - loss: 137.6402 - val_loss: 117.3898
Epoch 316/10000
5/5 [==============================] - 0s 7ms/step - loss: 136.7343 - val_loss: 187.1544
Epoch 317/10000
5/5 [==============================] - 0s 5ms/step - loss: 138.5453 - val_loss: 128.1528
Epoch 318/10000
5/5 [==============================] - 0s 6ms/step - loss: 133.7673 - val_loss: 125.8750
Epoch 319/10000
5/5 [==============================] - 0s 5ms/step - loss: 131.2880 - val_loss: 178.6842
Epoch 320/10000
5/5 [==============================] - 0s 6ms/step - loss: 152.3224 - val_loss: 110.2299
Epoch 321/10000
5/5 [==============================] - 0s 6ms/step - loss: 132.1650 - val_loss: 136.7408
Epoch 322/10000
5/5 [==============================] - 0s 7ms/step - lo

...

5/5 [==============================] - 0s 6ms/step - loss: 165.1177 - val_loss: 113.4639
Epoch 1170/10000
5/5 [==============================] - 0s 6ms/step - loss: 129.5469 - val_loss: 174.2698
Epoch 1171/10000
5/5 [==============================] - 0s 6ms/step - loss: 146.2059 - val_loss: 145.1059
Epoch 1172/10000
5/5 [==============================] - 0s 6ms/step - loss: 119.8856 - val_loss: 164.6861
Epoch 1173/10000
5/5 [==============================] - 0s 5ms/step - loss: 126.3762 - val_loss: 172.0092
Epoch 1174/10000
5/5 [==============================] - 0s 5ms/step - loss: 100.6522 - val_loss: 161.5015
Epoch 1175/10000
5/5 [==============================] - 0s 5ms/step - loss: 115.0910 - val_loss: 185.8884
Epoch 1176/10000
5/5 [==============================] - 0s 5ms/step - loss: 86.2393 - val_loss: 197.4422
Epoch 1177/10000
5/5 [==============================] - 0s 5ms/step - loss: 87.3692 - val_loss: 300.8357
Epoch 1178/10000
5/5 [==============================] - 0s 5ms/st

5/5 [==============================] - 0s 5ms/step - loss: 106.8713 - val_loss: 164.4575
Epoch 1247/10000
5/5 [==============================] - 0s 5ms/step - loss: 110.9377 - val_loss: 150.4941
Epoch 1248/10000
5/5 [==============================] - 0s 5ms/step - loss: 115.3123 - val_loss: 207.4978
Epoch 1249/10000
5/5 [==============================] - 0s 5ms/step - loss: 109.2280 - val_loss: 182.1020
Epoch 1250/10000
5/5 [==============================] - 0s 6ms/step - loss: 108.3758 - val_loss: 172.4947
Epoch 1251/10000
5/5 [==============================] - 0s 6ms/step - loss: 102.1507 - val_loss: 164.0880
Epoch 1252/10000
5/5 [==============================] - 0s 5ms/step - loss: 98.0513 - val_loss: 149.9314
Epoch 1253/10000
5/5 [==============================] - 0s 5ms/step - loss: 98.9142 - val_loss: 184.3367
Epoch 1254/10000
5/5 [==============================] - 0s 5ms/step - loss: 111.6642 - val_loss: 176.5805
Epoch 1255/10000
5/5 [==============================] - 0s 5ms/st

5/5 [==============================] - 0s 6ms/step - loss: 99.1349 - val_loss: 139.5403
Epoch 1324/10000
5/5 [==============================] - 0s 5ms/step - loss: 95.8223 - val_loss: 135.0615
Epoch 1325/10000
5/5 [==============================] - 0s 6ms/step - loss: 92.6512 - val_loss: 193.5615
Epoch 1326/10000
5/5 [==============================] - 0s 6ms/step - loss: 93.2088 - val_loss: 196.8298
Epoch 1327/10000
5/5 [==============================] - 0s 5ms/step - loss: 99.3985 - val_loss: 196.2277
Epoch 1328/10000
5/5 [==============================] - 0s 5ms/step - loss: 94.9095 - val_loss: 229.8656
Epoch 1329/10000
5/5 [==============================] - 0s 5ms/step - loss: 100.3920 - val_loss: 181.7859
Epoch 1330/10000
5/5 [==============================] - 0s 5ms/step - loss: 97.6436 - val_loss: 190.5266
Epoch 1331/10000
5/5 [==============================] - 0s 5ms/step - loss: 98.5919 - val_loss: 200.8170
Epoch 1332/10000
5/5 [==============================] - 0s 5ms/step - l

Epoch 1401/10000
5/5 [==============================] - 0s 7ms/step - loss: 100.9696 - val_loss: 182.2289
Epoch 1402/10000
5/5 [==============================] - 0s 5ms/step - loss: 110.8270 - val_loss: 277.0882
Epoch 1403/10000
5/5 [==============================] - 0s 6ms/step - loss: 129.8963 - val_loss: 185.9775
Epoch 1404/10000
5/5 [==============================] - 0s 6ms/step - loss: 104.8701 - val_loss: 138.0337
Epoch 1405/10000
5/5 [==============================] - 0s 6ms/step - loss: 121.2255 - val_loss: 112.7759
Epoch 1406/10000
5/5 [==============================] - 0s 6ms/step - loss: 118.0454 - val_loss: 111.4722
Epoch 1407/10000
5/5 [==============================] - 0s 6ms/step - loss: 115.5180 - val_loss: 134.9939
Epoch 1408/10000
5/5 [==============================] - 0s 6ms/step - loss: 119.6529 - val_loss: 145.2676
Epoch 1409/10000
5/5 [==============================] - 0s 5ms/step - loss: 123.1478 - val_loss: 153.2271
Epoch 1410/10000
5/5 [========================

5/5 [==============================] - 0s 5ms/step - loss: 107.5062 - val_loss: 198.8039
Epoch 1479/10000
5/5 [==============================] - 0s 6ms/step - loss: 115.0577 - val_loss: 185.2336
Epoch 1480/10000
5/5 [==============================] - 0s 6ms/step - loss: 98.6944 - val_loss: 198.4732
Epoch 1481/10000
5/5 [==============================] - 0s 9ms/step - loss: 97.3759 - val_loss: 182.8431
Epoch 1481: early stopping


In [19]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [20]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,92.779861,92.764565,92.719574,92.55719,80.586494,78.824364,79.213287,78.585823,80.060463,81.054634,83.144043,91.911934,82.901215,81.475563,92.801308,91.332153,96.144127,81.36525,79.375511,92.53215,92.043335,91.589127,97.542534,97.686256,97.724495,97.724503,97.72448,97.724472,97.724358,89.772186
Target,120.055,125.769,112.904,123.201,112.725,90.828,96.36,86.444,94.285,98.986,88.072,141.01,122.652,142.145,121.124,130.503,104.115,90.69,102.685,96.144,102.197,106.712,124.057,124.625,133.116,144.31,140.357,152.769,124.038,95.054
Error,27.275139,33.004433,20.184425,30.643806,32.138504,12.003639,17.146713,7.858177,14.224541,17.931366,4.927956,49.098061,39.750786,60.669441,28.322693,39.170853,7.970871,9.324753,23.309486,3.611847,10.153664,15.122871,26.514465,26.938744,35.391502,46.585495,42.632515,55.044525,26.313644,5.281815


In [21]:
display(mae)
display(mape)

25.618225

0.2086544

In [22]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [23]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 18ms/step
Ano-0: |Prediction[[1024.2023]] - Target[1290.639]| =  Error: [[266.43677]]; MAPE:[[0.20643787]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[1076.7885]] - Target[1367.6490000000001]| =  Error: [[290.8606]]; MAPE:[[0.21267195]]
1/1 [==============================] - 0s 17ms/step
Ano-5: |Prediction[[578.39453]] - Target[789.644]| =  Error: [[211.24945]]; MAPE:[[0.26752493]]


[array([[266.43677]], dtype=float32),
 array([[290.8606]], dtype=float32),
 array([[211.24945]], dtype=float32)]

256.18228

0.22887826